In [1]:
import sys, os, pickle

sys.path.insert(0, '/home/jupyter/AdversarialRobustness')
sys.path.insert(0, '/home/jupyter/AdversarialRobustness/vim')

# Add sbin to path for Trigon - fix of common Debian error
original_path = os.environ.get('PATH')
os.environ['PATH'] = original_path + ':/sbin'

import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import requests
from matplotlib import pyplot as plt
from torchvision import datasets
from transformers import ViTImageProcessor, ViTForImageClassification

from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

import loadVim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def save_checkpoint(state, filename="checkpoint.pth.tar"):
    torch.save(state, filename)

def load_checkpoint(filename="checkpoint.pth.tar", device=torch.device('cuda')):
    checkpoint = torch.load(filename, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch = checkpoint['epoch']
    return model, optimizer, epoch, checkpoint.get('loss', 0)

In [3]:
def compute_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1) 
    correct = torch.sum(preds == labels).item() 
    return correct / len(labels)

In [4]:
def validate(model, valid_loader, criterion, device):
    model.eval()
    total_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs.logits, labels)

            accuracy = compute_accuracy(outputs.logits, labels)
            total_accuracy += accuracy
            total_loss += loss.item()
    
    average_loss = total_loss / len(valid_loader)
    average_accuracy = total_accuracy / len(valid_loader)
    return average_loss, average_accuracy

# Training Loop

In [5]:
def train(model, train_loader, valid_loader, optimizer, criterion, device, num_epochs, pathmodel):
    logs = []
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        total_accuracy = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs.logits, labels)

            accuracy = compute_accuracy(outputs.logits, labels)
            total_accuracy += accuracy

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        
        average_loss = total_loss / len(train_loader)
        average_accuracy = total_accuracy / len(train_loader)
        val_loss, val_accuracy = validate(model, valid_loader, criterion, device)
        
        logs.append({"train":{"loss":average_loss, "acc":average_accuracy}, "val":{"loss":val_loss, "acc":val_accuracy}})

        print(f"Epoch {epoch+1}, Train Loss: {average_loss:.4f}, Train Accuracy: {average_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    save_checkpoint({'state_dict': model.state_dict(), 'log':logs}, filename=pathmodel)

In [6]:
def trainWrap(pathtrain, pathval, pathmodel, nepochs, model="vim"):
    if model == "vim":
        model, processor = loadVim.prepareDownstreamVim()
    elif model == "vit":
        model, processor = loadVim.prepareDownstreamVit()
    else: # ResNet
        model, processor = loadVim.prepareDownstreamResnet()

    transform = transforms.Compose([
        lambda image: processor(images=image, return_tensors='pt')['pixel_values'].squeeze(0)
    ])

    datasetTrain = datasets.ImageFolder(pathtrain, transform = transform)
    datasetVal = datasets.ImageFolder(pathval, transform = transform)

    loaderTrain = DataLoader(datasetTrain, batch_size=128, shuffle=True)
    loaderVal = DataLoader(datasetVal, batch_size=128, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    criterion = torch.nn.CrossEntropyLoss()
    model = model.to(device)
    
    train(model, loaderTrain, loaderVal, optimizer, criterion, device, nepochs, pathmodel)

# Vim Tuning

In [8]:
pathtrain = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebTrainA"
pathval = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"
pathmodel = "/home/jupyter/AdversarialRobustness/models/VimA.pth"

trainWrap(pathtrain, pathval, pathmodel, 20)
# state_dict and log saved in dictionary at pathmodel.

Epoch 1, Train Loss: 1.9095, Train Accuracy: 0.2878, Val Loss: 1.3994, Val Accuracy: 0.4831
Epoch 2, Train Loss: 1.3237, Train Accuracy: 0.5229, Val Loss: 1.1946, Val Accuracy: 0.5733
Epoch 3, Train Loss: 1.1047, Train Accuracy: 0.5853, Val Loss: 1.1789, Val Accuracy: 0.5279
Epoch 4, Train Loss: 1.0166, Train Accuracy: 0.6400, Val Loss: 1.0358, Val Accuracy: 0.6000
Epoch 5, Train Loss: 0.8842, Train Accuracy: 0.6776, Val Loss: 1.0370, Val Accuracy: 0.5647
Epoch 6, Train Loss: 0.8506, Train Accuracy: 0.6917, Val Loss: 0.9461, Val Accuracy: 0.6748
Epoch 7, Train Loss: 0.7604, Train Accuracy: 0.7245, Val Loss: 0.9037, Val Accuracy: 0.6864
Epoch 8, Train Loss: 0.7960, Train Accuracy: 0.6949, Val Loss: 1.0110, Val Accuracy: 0.6186
Epoch 9, Train Loss: 0.7697, Train Accuracy: 0.7079, Val Loss: 0.9418, Val Accuracy: 0.6660
Epoch 10, Train Loss: 0.7355, Train Accuracy: 0.7115, Val Loss: 0.9198, Val Accuracy: 0.6702
Epoch 11, Train Loss: 0.7267, Train Accuracy: 0.7364, Val Loss: 0.9789, Val Acc

In [9]:
pathtrain = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebTrainB"
pathval = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"
pathmodel = "/home/jupyter/AdversarialRobustness/models/VimB.pth"

trainWrap(pathtrain, pathval, pathmodel, 20)

Epoch 1, Train Loss: 1.8283, Train Accuracy: 0.3146, Val Loss: 1.3761, Val Accuracy: 0.5262
Epoch 2, Train Loss: 1.2348, Train Accuracy: 0.5428, Val Loss: 1.1845, Val Accuracy: 0.5520
Epoch 3, Train Loss: 0.9911, Train Accuracy: 0.6424, Val Loss: 1.0807, Val Accuracy: 0.5804
Epoch 4, Train Loss: 0.8882, Train Accuracy: 0.6588, Val Loss: 1.0678, Val Accuracy: 0.5999
Epoch 5, Train Loss: 0.8155, Train Accuracy: 0.7176, Val Loss: 1.0476, Val Accuracy: 0.6122
Epoch 6, Train Loss: 0.7544, Train Accuracy: 0.7310, Val Loss: 0.9935, Val Accuracy: 0.6220
Epoch 7, Train Loss: 0.6427, Train Accuracy: 0.7776, Val Loss: 1.0437, Val Accuracy: 0.6252
Epoch 8, Train Loss: 0.6508, Train Accuracy: 0.7823, Val Loss: 0.9869, Val Accuracy: 0.6415
Epoch 9, Train Loss: 0.6308, Train Accuracy: 0.7908, Val Loss: 1.0071, Val Accuracy: 0.6095
Epoch 10, Train Loss: 0.5679, Train Accuracy: 0.8289, Val Loss: 1.0041, Val Accuracy: 0.6435
Epoch 11, Train Loss: 0.5906, Train Accuracy: 0.8062, Val Loss: 0.9721, Val Acc

In [10]:
# Accuracies in hindsight:

for dataset in ["A", "B"]:
    mod = torch.load(f"/home/jupyter/AdversarialRobustness/models/Vim{dataset}.pth")
    model, processor = loadVim.prepareDownstreamVim()
    model.vim_model.load_state_dict(mod["state_dict"])
    transform = transforms.Compose([
            lambda image: processor(images=image, return_tensors='pt')['pixel_values'].squeeze(0)
        ])
    datasetTest = datasets.ImageFolder("/home/jupyter/AdversarialRobustness/CelebSubset/CelebTest", transform = transform)
    loaderTrain = DataLoader(datasetTest, batch_size=128, shuffle=False)
    print("Results for dataset ", dataset)
    print(validate(model.to(device), loaderTrain, torch.nn.CrossEntropyLoss(), device))

Results for dataset  A
(0.9388849437236786, 0.6850490196078431)
Results for dataset  B
(0.9405563771724701, 0.7212775735294117)


# ViT Tuning

In [8]:
pathtrain = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebTrainA"
pathval = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"
pathmodel = "/home/jupyter/AdversarialRobustness/models/VitA.pth"

trainWrap(pathtrain, pathval, pathmodel, 20, model="vit")

Epoch 1, Train Loss: 1.2401, Train Accuracy: 0.5301, Val Loss: 0.6455, Val Accuracy: 0.7558
Epoch 2, Train Loss: 0.4754, Train Accuracy: 0.8230, Val Loss: 0.6222, Val Accuracy: 0.7433
Epoch 3, Train Loss: 0.3282, Train Accuracy: 0.8707, Val Loss: 0.3725, Val Accuracy: 0.8666
Epoch 4, Train Loss: 0.2411, Train Accuracy: 0.9087, Val Loss: 0.3697, Val Accuracy: 0.8852
Epoch 5, Train Loss: 0.1716, Train Accuracy: 0.9317, Val Loss: 0.3260, Val Accuracy: 0.8715
Epoch 6, Train Loss: 0.0969, Train Accuracy: 0.9766, Val Loss: 0.3564, Val Accuracy: 0.8668
Epoch 7, Train Loss: 0.0828, Train Accuracy: 0.9837, Val Loss: 0.3383, Val Accuracy: 0.8579
Epoch 8, Train Loss: 0.0572, Train Accuracy: 0.9941, Val Loss: 0.3138, Val Accuracy: 0.8726
Epoch 9, Train Loss: 0.0463, Train Accuracy: 0.9961, Val Loss: 0.2909, Val Accuracy: 0.8794
Epoch 10, Train Loss: 0.0389, Train Accuracy: 0.9980, Val Loss: 0.2849, Val Accuracy: 0.8881
Epoch 11, Train Loss: 0.0327, Train Accuracy: 0.9990, Val Loss: 0.2908, Val Acc

In [9]:
pathtrain = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebTrainB"
pathval = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"
pathmodel = "/home/jupyter/AdversarialRobustness/models/VitB.pth"

trainWrap(pathtrain, pathval, pathmodel, 20, model="vit")

Epoch 1, Train Loss: 1.3098, Train Accuracy: 0.4871, Val Loss: 0.5895, Val Accuracy: 0.7819
Epoch 2, Train Loss: 0.4890, Train Accuracy: 0.8325, Val Loss: 0.4905, Val Accuracy: 0.8491
Epoch 3, Train Loss: 0.3087, Train Accuracy: 0.8855, Val Loss: 0.4881, Val Accuracy: 0.8412
Epoch 4, Train Loss: 0.1912, Train Accuracy: 0.9395, Val Loss: 0.4139, Val Accuracy: 0.8434
Epoch 5, Train Loss: 0.1379, Train Accuracy: 0.9514, Val Loss: 0.4069, Val Accuracy: 0.8802
Epoch 6, Train Loss: 0.1204, Train Accuracy: 0.9661, Val Loss: 0.3859, Val Accuracy: 0.8647
Epoch 7, Train Loss: 0.0803, Train Accuracy: 0.9834, Val Loss: 0.3827, Val Accuracy: 0.8686
Epoch 8, Train Loss: 0.0675, Train Accuracy: 0.9902, Val Loss: 0.3969, Val Accuracy: 0.8783
Epoch 9, Train Loss: 0.0479, Train Accuracy: 0.9971, Val Loss: 0.3833, Val Accuracy: 0.8958
Epoch 10, Train Loss: 0.0405, Train Accuracy: 0.9971, Val Loss: 0.3773, Val Accuracy: 0.8997
Epoch 11, Train Loss: 0.0319, Train Accuracy: 0.9980, Val Loss: 0.3932, Val Acc

In [10]:
# Accuracies in hindsight:

for dataset in ["A", "B"]:
    mod = torch.load(f"/home/jupyter/AdversarialRobustness/models/Vit{dataset}.pth")
    model, processor = loadVim.prepareDownstreamVit()
    model.load_state_dict(mod["state_dict"])
    transform = transforms.Compose([
            lambda image: processor(images=image, return_tensors='pt')['pixel_values'].squeeze(0)
        ])
    datasetTest = datasets.ImageFolder("/home/jupyter/AdversarialRobustness/CelebSubset/CelebTest", transform = transform)
    loaderTrain = DataLoader(datasetTest, batch_size=128, shuffle=False)
    print("Results for dataset ", dataset)
    print(validate(model.to(device), loaderTrain, torch.nn.CrossEntropyLoss(), device))

Results for dataset  A
(0.2904525399208069, 0.9129136029411764)
Results for dataset  B
(0.2982930690050125, 0.8953737745098039)


# ResNet Tuning

In [7]:
pathtrain = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebTrainA"
pathval = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"
pathmodel = "/home/jupyter/AdversarialRobustness/models/ResNetA.pth"

trainWrap(pathtrain, pathval, pathmodel, 20, model="resnet")

Epoch 1, Train Loss: 2.2684, Train Accuracy: 0.2126, Val Loss: 2.1682, Val Accuracy: 0.2023
Epoch 2, Train Loss: 1.9666, Train Accuracy: 0.3271, Val Loss: 1.9724, Val Accuracy: 0.3989
Epoch 3, Train Loss: 1.6519, Train Accuracy: 0.4933, Val Loss: 1.9446, Val Accuracy: 0.2897
Epoch 4, Train Loss: 1.4403, Train Accuracy: 0.5717, Val Loss: 1.8794, Val Accuracy: 0.2990
Epoch 5, Train Loss: 1.3350, Train Accuracy: 0.6254, Val Loss: 1.7883, Val Accuracy: 0.4339
Epoch 6, Train Loss: 1.1973, Train Accuracy: 0.6769, Val Loss: 1.6809, Val Accuracy: 0.4661
Epoch 7, Train Loss: 1.0844, Train Accuracy: 0.7212, Val Loss: 1.5516, Val Accuracy: 0.4673
Epoch 8, Train Loss: 0.9862, Train Accuracy: 0.7774, Val Loss: 1.5108, Val Accuracy: 0.5157
Epoch 9, Train Loss: 0.9119, Train Accuracy: 0.7832, Val Loss: 1.4056, Val Accuracy: 0.5443
Epoch 10, Train Loss: 0.9064, Train Accuracy: 0.7733, Val Loss: 1.4476, Val Accuracy: 0.4985
Epoch 11, Train Loss: 0.8286, Train Accuracy: 0.7996, Val Loss: 1.3525, Val Acc

In [8]:
pathtrain = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebTrainB"
pathval = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"
pathmodel = "/home/jupyter/AdversarialRobustness/models/ResNetB.pth"

trainWrap(pathtrain, pathval, pathmodel, 20, model="resnet")

Epoch 1, Train Loss: 2.3905, Train Accuracy: 0.1545, Val Loss: 2.2187, Val Accuracy: 0.0664
Epoch 2, Train Loss: 2.0348, Train Accuracy: 0.3119, Val Loss: 2.0542, Val Accuracy: 0.2388
Epoch 3, Train Loss: 1.6819, Train Accuracy: 0.4940, Val Loss: 2.0122, Val Accuracy: 0.3379
Epoch 4, Train Loss: 1.4845, Train Accuracy: 0.5670, Val Loss: 1.8730, Val Accuracy: 0.4146
Epoch 5, Train Loss: 1.3537, Train Accuracy: 0.5814, Val Loss: 1.7994, Val Accuracy: 0.4546
Epoch 6, Train Loss: 1.2529, Train Accuracy: 0.6373, Val Loss: 1.7535, Val Accuracy: 0.2872
Epoch 7, Train Loss: 1.1850, Train Accuracy: 0.6561, Val Loss: 1.6322, Val Accuracy: 0.4267
Epoch 8, Train Loss: 1.0656, Train Accuracy: 0.7366, Val Loss: 1.4832, Val Accuracy: 0.5041
Epoch 9, Train Loss: 0.9824, Train Accuracy: 0.7503, Val Loss: 1.4024, Val Accuracy: 0.5181
Epoch 10, Train Loss: 0.9351, Train Accuracy: 0.7882, Val Loss: 1.3464, Val Accuracy: 0.6093
Epoch 11, Train Loss: 0.8360, Train Accuracy: 0.8172, Val Loss: 1.3688, Val Acc

In [11]:
# Accuracies in hindsight:

for dataset in ["A", "B"]:
    mod = torch.load(f"/home/jupyter/AdversarialRobustness/models/ResNet{dataset}.pth")
    model, processor = loadVim.prepareDownstreamResnet()
    model.load_state_dict(mod["state_dict"])
    transform = transforms.Compose([
            lambda image: processor(images=image, return_tensors='pt')['pixel_values'].squeeze(0)
        ])
    datasetTest = datasets.ImageFolder("/home/jupyter/AdversarialRobustness/CelebSubset/CelebTest", transform = transform)
    loaderTrain = DataLoader(datasetTest, batch_size=128, shuffle=False)
    print("Results for dataset ", dataset)
    print(validate(model.to(device), loaderTrain, torch.nn.CrossEntropyLoss(), device))

Results for dataset  A
(1.3444638848304749, 0.5605851715686274)
Results for dataset  B
(1.2706778049468994, 0.5519301470588236)


# Model Hardening with Adversarial Training

Only model variants with suffix A will be used here.

In [15]:
def advTrainWrap(pathtrain, pathtrainreg, pathval, pathmodel, pathcheckpoint, nepochs, model="vim"):
    if model == "vim":
        model, processor = loadVim.prepareDownstreamVim()
        checkpoint = torch.load(pathcheckpoint)
        model.vim_model.load_state_dict(checkpoint["state_dict"])
    elif model == "vit":
        model, processor = loadVim.prepareDownstreamVit()
        checkpoint = torch.load(pathcheckpoint)
        model.load_state_dict(checkpoint["state_dict"])
    else: # ResNet
        model, processor = loadVim.prepareDownstreamResnet()
        checkpoint = torch.load(pathcheckpoint)
        model.load_state_dict(checkpoint["state_dict"])

    transform = transforms.Compose([
        lambda image: processor(images=image, return_tensors='pt')['pixel_values'].squeeze(0)
    ])
    
    def preproc(path):
        tensor = torch.load(path)
        return tensor.squeeze(0).cpu()
    
    datasetTrainReg = datasets.ImageFolder(pathtrainreg, transform = transform)
    datasetTrainAdv = datasets.DatasetFolder(pathtrain, loader = preproc, extensions=".pt")
    # Combine them:
    datasetTrain = torch.utils.data.ConcatDataset([datasetTrainReg, datasetTrainAdv])
    
    datasetVal = datasets.ImageFolder(pathval, transform = transform)

    loaderTrain = DataLoader(datasetTrain, batch_size=128, shuffle=True)
    loaderVal = DataLoader(datasetVal, batch_size=128, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)
    criterion = torch.nn.CrossEntropyLoss()
    model = model.to(device)
    
    train(model, loaderTrain, loaderVal, optimizer, criterion, device, nepochs, pathmodel)

## Vim Adversarial Training

In [12]:
pathtrainadv = "/home/jupyter/AdversarialRobustness/CelebAdv/CelAdvVimA"
pathtrainreg = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"
pathval = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebTest"
pathcheckpoint = "/home/jupyter/AdversarialRobustness/models/VimA.pth"
pathmodel = "/home/jupyter/AdversarialRobustness/models/AdvVimA.pth"

advTrainWrap(pathtrainadv, pathtrainreg, pathval, pathmodel, pathcheckpoint, 20)

Epoch 1, Train Loss: 1.6491, Train Accuracy: 0.5050, Val Loss: 0.9528, Val Accuracy: 0.6968
Epoch 2, Train Loss: 1.3516, Train Accuracy: 0.5741, Val Loss: 0.9381, Val Accuracy: 0.7105
Epoch 3, Train Loss: 1.1850, Train Accuracy: 0.5896, Val Loss: 0.9263, Val Accuracy: 0.6998
Epoch 4, Train Loss: 1.0310, Train Accuracy: 0.6703, Val Loss: 0.9064, Val Accuracy: 0.7066
Epoch 5, Train Loss: 0.8558, Train Accuracy: 0.7126, Val Loss: 0.9130, Val Accuracy: 0.7016
Epoch 6, Train Loss: 0.8606, Train Accuracy: 0.7012, Val Loss: 0.9350, Val Accuracy: 0.7046
Epoch 7, Train Loss: 0.7901, Train Accuracy: 0.7242, Val Loss: 0.9319, Val Accuracy: 0.7144
Epoch 8, Train Loss: 0.6910, Train Accuracy: 0.7619, Val Loss: 0.9484, Val Accuracy: 0.6919
Epoch 9, Train Loss: 0.6079, Train Accuracy: 0.8042, Val Loss: 0.9446, Val Accuracy: 0.6772
Epoch 10, Train Loss: 0.5786, Train Accuracy: 0.7887, Val Loss: 0.9624, Val Accuracy: 0.6810
Epoch 11, Train Loss: 0.5718, Train Accuracy: 0.7893, Val Loss: 0.9529, Val Acc

## Vit Adversarial Training

In [16]:
pathtrainadv = "/home/jupyter/AdversarialRobustness/CelebAdv/CelAdvVitA"
pathtrainreg = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"
pathval = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebTest"
pathcheckpoint = "/home/jupyter/AdversarialRobustness/models/VitA.pth"
pathmodel = "/home/jupyter/AdversarialRobustness/models/AdvVitA.pth"

advTrainWrap(pathtrainadv, pathtrainreg, pathval, pathmodel, pathcheckpoint, 20, model = "vit")

Epoch 1, Train Loss: 1.7127, Train Accuracy: 0.6198, Val Loss: 0.5507, Val Accuracy: 0.8788
Epoch 2, Train Loss: 0.9574, Train Accuracy: 0.7124, Val Loss: 0.4036, Val Accuracy: 0.8876
Epoch 3, Train Loss: 0.6691, Train Accuracy: 0.7925, Val Loss: 0.3389, Val Accuracy: 0.8973
Epoch 4, Train Loss: 0.4581, Train Accuracy: 0.8541, Val Loss: 0.3483, Val Accuracy: 0.8904
Epoch 5, Train Loss: 0.3326, Train Accuracy: 0.8854, Val Loss: 0.3798, Val Accuracy: 0.8757
Epoch 6, Train Loss: 0.2484, Train Accuracy: 0.9208, Val Loss: 0.3811, Val Accuracy: 0.8728
Epoch 7, Train Loss: 0.2018, Train Accuracy: 0.9442, Val Loss: 0.4121, Val Accuracy: 0.8856
Epoch 8, Train Loss: 0.1385, Train Accuracy: 0.9648, Val Loss: 0.4303, Val Accuracy: 0.8611
Epoch 9, Train Loss: 0.1275, Train Accuracy: 0.9682, Val Loss: 0.4189, Val Accuracy: 0.8611
Epoch 10, Train Loss: 0.0829, Train Accuracy: 0.9902, Val Loss: 0.3880, Val Accuracy: 0.8895
Epoch 11, Train Loss: 0.0752, Train Accuracy: 0.9863, Val Loss: 0.3884, Val Acc

## ResNet Adversarial Training

In [17]:
pathtrainadv = "/home/jupyter/AdversarialRobustness/CelebAdv/CelAdvResNetA"
pathtrainreg = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"
pathval = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebTest"
pathcheckpoint = "/home/jupyter/AdversarialRobustness/models/ResNetA.pth"
pathmodel = "/home/jupyter/AdversarialRobustness/models/AdvResNetA.pth"

advTrainWrap(pathtrainadv, pathtrainreg, pathval, pathmodel, pathcheckpoint, 20, model = "resnet")

Epoch 1, Train Loss: 1.2936, Train Accuracy: 0.5535, Val Loss: 1.3707, Val Accuracy: 0.5381
Epoch 2, Train Loss: 1.1756, Train Accuracy: 0.6051, Val Loss: 1.3476, Val Accuracy: 0.5478
Epoch 3, Train Loss: 1.0636, Train Accuracy: 0.6557, Val Loss: 1.3081, Val Accuracy: 0.5596
Epoch 4, Train Loss: 0.9809, Train Accuracy: 0.7053, Val Loss: 1.2788, Val Accuracy: 0.5733
Epoch 5, Train Loss: 0.9548, Train Accuracy: 0.7282, Val Loss: 1.2527, Val Accuracy: 0.5890
Epoch 6, Train Loss: 0.8591, Train Accuracy: 0.7526, Val Loss: 1.2947, Val Accuracy: 0.5734
Epoch 7, Train Loss: 0.8057, Train Accuracy: 0.7789, Val Loss: 1.2994, Val Accuracy: 0.5626
Epoch 8, Train Loss: 0.7249, Train Accuracy: 0.8258, Val Loss: 1.2880, Val Accuracy: 0.5665
Epoch 9, Train Loss: 0.7032, Train Accuracy: 0.8346, Val Loss: 1.2829, Val Accuracy: 0.5655
Epoch 10, Train Loss: 0.6575, Train Accuracy: 0.8576, Val Loss: 1.2760, Val Accuracy: 0.5597
Epoch 11, Train Loss: 0.6070, Train Accuracy: 0.8748, Val Loss: 1.2819, Val Acc